In [7]:
import pathlib

import awkward as ak
import librosa
import numpy as np
import pandas as pd
import scipy
import torch
from tqdm import tqdm

from utils import get_file_list, import_audio_dataset, encode_file_names, \
    pad_awkward_array, audio_tokenizer, downsample_tensor, get_file_list

In [19]:
Xr, srr = import_audio_dataset("./ravdess/", ".wav")
Xc, src = import_audio_dataset("./crema/", ".wav")

2024-03-19 19:47:13.849 | INFO     | utils:import_audio_dataset:71 - Loading 2452 files to list...
  0%|          | 0/2452 [00:00<?, ?it/s]

100%|██████████| 2452/2452 [00:09<00:00, 263.90it/s]
2024-03-19 19:47:23.149 | INFO     | utils:import_audio_dataset:78 - Loading 2452 files to awkward array...
100%|██████████| 2452/2452 [06:17<00:00,  6.50it/s]
2024-03-19 19:53:49.011 | INFO     | utils:import_audio_dataset:71 - Loading 7442 files to list...
100%|██████████| 7442/7442 [00:08<00:00, 853.94it/s] 
2024-03-19 19:53:57.734 | INFO     | utils:import_audio_dataset:78 - Loading 7442 files to awkward array...
100%|██████████| 7442/7442 [05:55<00:00, 20.92it/s]


In [21]:
Xr = pad_awkward_array(Xr)

In [20]:
Xc = pad_awkward_array(Xc)

In [25]:
Xc = torch.tensor(Xc.to_numpy())

In [27]:
Xr = torch.tensor(Xr.to_numpy())

In [29]:
Xr = downsample_tensor(Xr, 48000, 16000)

In [30]:
Xr = audio_tokenizer(Xr, 16000, 27.5)
Xc = audio_tokenizer(Xc, 16000, 27.5)

2024-03-19 20:46:36.360 | INFO     | utils:audio_tokenizer:283 - Token size 440; Padded length 102080
;Number of tokens 232
2024-03-19 20:46:37.182 | INFO     | utils:audio_tokenizer:283 - Token size 440; Padded length 80080
;Number of tokens 182


In [32]:
Yr = encode_file_names(get_file_list("./ravdess/", ".wav"), "ravdess")
Yc = encode_file_names(get_file_list("./crema/", ".wav"), "crema")

In [33]:
Yr_sorted_actor = Yr.sort_values(by=["actor", "filename"])

In [34]:
Yr_train = Yr_sorted_actor[Yr_sorted_actor.actor.astype(int) < 19]
Yr_vt = Yr_sorted_actor[Yr_sorted_actor.actor.astype(int) >= 19]

In [35]:
Yr_val = Yr_vt[Yr_vt.actor.astype(int)<22]
Yr_test = Yr_vt[Yr_vt.actor.astype(int)>=22]

In [36]:
idx_train = list(Yr_train.index)
idx_val = list(Yr_val.index)
idx_test = list(Yr_test.index)

In [51]:
rpldict = dict(zip(sorted(np.unique(Yr_train.emotion)), range(0,8)))
y_train = Yr_train.emotion.replace(rpldict).values
y_val = Yr_val.emotion.replace(rpldict).values
y_test = Yr_test.emotion.replace(rpldict).values

In [52]:
Xr_train = Xr[idx_train]
Xr_val = Xr[idx_val]
Xr_test = Xr[idx_test]

In [53]:
Yc_sorted_actor = Yc.sort_values(by=["actor", "filename"])

In [55]:
Yc_train = Yc_sorted_actor[Yc_sorted_actor.actor.astype(int) < 72]
Yc_val = Yc_sorted_actor[Yc_sorted_actor.actor.astype(int) >= 72]

In [56]:
idx_train = list(Yc_train.index)
idx_val = list(Yc_val.index)

In [57]:
Xc_train = Xc[idx_train]
Xc_val = Xc[idx_val]

In [60]:
rpldict = dict(zip(sorted(np.unique(Yc_train.emotion)), range(0,6)))
yc_train = Yc_train.emotion.replace(rpldict).values
yc_val = Yc_val.emotion.replace(rpldict).values

In [61]:
torch.save(Xr_train, "./Xr_train.pt")
torch.save(Xr_test, "./Xr_test.pt")
torch.save(Xr_val, "./Xr_val.pt")

torch.save(Xc_train, "./Xc_train.pt")
torch.save(Xc_val, "./Xc_val.pt")

In [62]:
torch.save(torch.tensor(y_train), "./y_train.pt")
torch.save(torch.tensor(y_test), "./y_test.pt")
torch.save(torch.tensor(y_val), "./y_val.pt")

torch.save(torch.tensor(yc_train), "./yc_train.pt")
torch.save(torch.tensor(yc_val), "./yc_val.pt")